# Task 1: Visualizing Gradient Descent Algorithms

This notebook explores and visualizes the behavior of several fundamental optimization algorithms used in machine learning. We will compare:

1.  **Vanilla Gradient Descent Variants:**
    *   Batch Gradient Descent
    *   Stochastic Gradient Descent (SGD)
    *   Mini-Batch Gradient Descent
2.  **Momentum-Based Variants:**
    *   Batch GD with Momentum
    *   SGD with Momentum
    *   Mini-Batch GD with Momentum

Our goal is to fit a simple linear regression model and observe how each algorithm navigates the loss landscape towards the optimal solution.

### The Math: Gradient Descent Update Rule

The core idea of gradient descent is to iteratively update the model parameters ($\theta$) in the opposite direction of the gradient of the loss function ($J(\theta)$).

**Standard Update Rule:**
$$ \theta_{t+1} = \theta_t - \eta \nabla J(\theta_t) $$
where $\eta$ is the learning rate.

**Update Rule with Momentum:**
Momentum introduces a velocity term ($v_t$) that accumulates gradients over time, helping to accelerate convergence and overcome local minima.
$$ v_{t+1} = \gamma v_t + \nabla J(\theta_t) $$
$$ \theta_{t+1} = \theta_t - \eta v_{t+1} $$
where $\gamma$ is the momentum coefficient.

In [1]:
import torch
import numpy as np
import sys
import os

# Add the src directory to the Python path
# This allows us to import our custom modules
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from training import train_linear_regression_gd
from visualize import generate_gradient_descent_gif

# Setup device for training
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

/Users/devanshlodha/Documents/github/signal-synth-ml/signal-synth-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: mps


## 1. Data Generation

We create a simple synthetic dataset based on a linear relationship with some added Gaussian noise. Our model will try to recover the original line `y = 3x + 4`.

In [2]:
# Set a seed for reproducibility
np.random.seed(45)
num_samples = 100

# Generate data: y = 3x + 4 + noise
x1_np = np.random.uniform(-2, 2, num_samples)
f_x = 3 * x1_np + 4
eps = np.random.randn(num_samples) * 1.5 # Add some noise
y_np = f_x + eps

# Convert to PyTorch tensors and move to the selected device
x = torch.tensor(x1_np, dtype=torch.float32, device=device)
y = torch.tensor(y_np, dtype=torch.float32, device=device)

## 2. Training and Visualization

We'll now run each optimization algorithm and generate a GIF to visualize its path on the loss contour plot. First, we need to compute the loss landscape to create the contour plot.

In [3]:
# Create the loss landscape for visualization
theta1_range = np.linspace(-2, 8, 100) # Range for slope (theta_1)
theta0_range = np.linspace(-2, 8, 100) # Range for intercept (theta_0)
SLOPE, INTERCEPT = np.meshgrid(theta1_range, theta0_range)

# Calculate loss for each point in the grid
loss_grid_np = np.zeros(SLOPE.shape)
for i in range(SLOPE.shape[0]):
    for j in range(SLOPE.shape[1]):
        slope_ij = SLOPE[i, j]
        intercept_ij = INTERCEPT[i, j]
        # Calculate Mean Squared Error
        loss_grid_np[i, j] = ((y.cpu().numpy() - x.cpu().numpy() * slope_ij - intercept_ij) ** 2).mean()

# We'll use a log scale for better visualization of the contours
loss_grid_for_plot = (INTERCEPT, SLOPE, np.log(loss_grid_np))

In [4]:
# Define hyperparameters for each algorithm
algorithms_to_run = {
    'batch': {'lr': 0.1, 'epochs': 50},
    'sgd': {'lr': 0.01, 'epochs': 5},
    'mini_batch': {'lr': 0.05, 'epochs': 20},
    'batch_momentum': {'lr': 0.1, 'epochs': 50, 'momentum': 0.7},
    'sgd_momentum': {'lr': 0.01, 'epochs': 5, 'momentum': 0.9},
    'mini_batch_momentum': {'lr': 0.05, 'epochs': 20, 'momentum': 0.8}
}

# Run the training and generate GIFs
for name, params in algorithms_to_run.items():
    print(f"--- Running {name.upper()} ---")
    final_t0, final_t1, history = train_linear_regression_gd(
        x=x,
        y=y,
        algorithm=name,
        learning_rate=params['lr'],
        epochs=params['epochs'],
        momentum=params.get('momentum', 0.9) # Default momentum if not specified
    )
    
    print(f"Final parameters: theta0={final_t0.item():.4f}, theta1={final_t1.item():.4f}")
    
    # Generate and save the GIF
    gif_path = f'../results/gifs/task1/{name}_descent.gif'
    generate_gradient_descent_gif(history, loss_grid_for_plot, gif_path)
    print("-" * 25 + "\n")

--- Running BATCH ---


Training with batch: 100%|██████████| 50/50 [00:00<00:00, 243.15it/s, loss=2.1947]


Final parameters: theta0=4.0512, theta1=2.9408


Generating GIF frames: 100%|██████████| 51/51 [00:01<00:00, 35.83it/s]


GIF saved to ../results/gifs/task1/batch_descent.gif
-------------------------

--- Running SGD ---


...

GIF saved to ../results/gifs/task1/sgd_momentum_descent.gif
-------------------------

--- Running MINI_BATCH_MOMENTUM ---


Training with mini_batch_momentum: 100%|██████████| 20/20 [00:00<00:00, 48.73it/s, loss=0.5091]


Final parameters: theta0=4.2261, theta1=2.8844


Generating GIF frames: 100%|██████████| 261/261 [00:06<00:00, 40.52it/s]


GIF saved to ../results/gifs/task1/mini_batch_momentum_descent.gif
-------------------------



## 3. Results and Conclusion

The generated GIFs are saved in the `results/gifs/task1/` directory. By observing them, we can compare the behaviors of the different algorithms.

### Batch Gradient Descent
![Batch GD](../results/gifs/task1/batch_descent.gif)

*Observes a smooth, direct path to the minimum. Computationally expensive per step as it uses the entire dataset.*

### Stochastic Gradient Descent (SGD)
![SGD](../results/gifs/task1/sgd_descent.gif)

*Observes a noisy, erratic path. Each step is very fast but the direction is less reliable, causing high variance in the parameter updates.*

### Mini-Batch Gradient Descent
![Mini-Batch GD](../results/gifs/task1/mini_batch_descent.gif)

*A good compromise. The path is less noisy than SGD, and convergence is much faster than standard Batch GD. This is the most common approach in deep learning.*

### Batch GD with Momentum
![Batch GD Momentum](../results/gifs/task1/batch_momentum_descent.gif)

*Momentum helps accelerate the descent, often overshooting the minimum slightly before settling. It's particularly effective in flatter regions of the loss landscape.*

### SGD with Momentum
![SGD Momentum](../results/gifs/task1/sgd_momentum_descent.gif)

*Momentum helps to dampen the oscillations of SGD. The accumulated gradient (velocity) provides a more stable direction, leading to faster convergence.*

### Mini-Batch GD with Momentum
![Mini-Batch GD Momentum](../results/gifs/task1/mini_batch_momentum_descent.gif)

*This combines the benefits of both mini-batching and momentum, often resulting in the fastest and most reliable convergence for a wide range of problems.*